In [34]:
import os
import math
import time
import pandas as pd
import simplekml
import shutil
from sys import platform
from pathlib import Path

# Measuring execution time
start_time = time.time()

In [35]:
# Checks what operating system the user is using
if platform == 'win32':
    root_path = r'C:\Users\Stefanos\DeLasCasas CP, LLc\Ro De Las Casas - ' \
                r'DLC2P Server\AA Stefanos\Data Science\Python Projects\CIS ' \
                r'Data to Google Earth (Real Time)'
elif platform == 'darwin':
    root_path = r'/Users/Stefanos/Library/CloudStorage/' \
                r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/Ro De Las Casas ' \
                r'- DLC2P Server/AA Stefanos/Data Science/Python Projects/' \
                r'CIS Data to Google Earth (Real Time)'

# Specify Variables

In [36]:
# SPECIFY SCALE FACTOR
scale_factor = 5
# SPECIFY TRANSPARENCY (0 to 1)
transparency = 1
# SPECIFY ICON SCALE
icon_scale = 0.3
# SPECIFY COLOR PALETTE
#my_cmap = plt.get_cmap('Spectral_r')
# PROJECT NAME
project_name = 'XYZ'    #input('\nPROJECT NAME:')

In [37]:
# Distance between points
#step_feet = df_cis['On Potential'].iloc[2]-df_cis['On Potential'].iloc[1]

In [38]:
#Convert miles to feet
#distance_feet = miles_per_kml*5280
# Number of data points
#data_points = int(distance_feet/step_feet)
# Total miles in the CIS survey (rounded up)

# Data Cleaning

In [39]:
# Creates a list of files in data folder
raw_data_list = os.listdir(r'/Users/Stefanos/Library/CloudStorage/'
                           r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/'
                           r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                           r'Data Science/Python Projects/'
                           r'CIS Data to Google Earth (Real Time)/'
                           r'Original Data')

In [40]:
# Create an empty list to exclude certain files
filtered_data_list = []
# Current list index
i = 0

# Filters the folder and excludes files that start with '.'
while i < len(raw_data_list):
    if not raw_data_list[i][0] == '.':
        filtered_data_list.append(raw_data_list[i])
    i += 1

In [41]:
# Creates an empty list to place the total miles
total_miles_list = []
# Current list index
i = 0

# Goes through all the files in the list
while i < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(root_path +
                         str(Path(r'/Original Data/' +
                                  filtered_data_list[i])))
    # Drops elevation and PDOP columns
    df_cis = df_cis.iloc[:, :6]
    # Delete rows that CIS was skipped
    df_cis = df_cis.loc[df_cis['On Potential'] != 'SKIP']
    # Convert object columns to numbers
    df_cis[['On Potential', 'Off Potential']] = \
                            df_cis[['On Potential', 'Off Potential']]\
                            .apply(pd.to_numeric, errors='coerce', axis=1)
    #
    total_miles_list.append(round(max(df_cis['Station'])/5280, 2))
    df_cis.to_csv(root_path +
                  str(Path(r'/Data/' +
                           filtered_data_list[i])), index=False)

    i += 1

# Create Directories (Folders)

In [42]:
# Delete old output folder and create a new one
if os.path.exists(root_path + r'/Output/'):
    shutil.rmtree(root_path + r'/Output/')
    os.makedirs(root_path + r'/Output/')
if not os.path.exists(root_path + r'/Output/'):
    os.makedirs(root_path + r'/Output/')

In [43]:
folder_mile = 1
i = 0

# Create CIS survey folders
while i < len(total_miles_list):
    # miles_remaining = total_miles_list[i]

    # while miles_remaining > 0:
    os.makedirs(root_path + r'/Output/' +
                filtered_data_list[i].split('.DAT', 1)[0] + '/')
        # folder_mile += 1
        # miles_remaining -= 1
        #
        # if 0 < miles_remaining < 1:
        #     folder_mile = total_miles_list[i]

    i += 1
    #folder_mile = 1

# CIS Data

## On

In [44]:
feet_counter = 5280
kml_mile = 1
i = 0       # Row for data
j = 0       # Row for CIS surveys

while j < len(filtered_data_list):
    # Goes through all CIS survey files
    df_cis_on = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Longitude', 'Latitude', 'On Potential']]\
        .reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential']*(-1)
    last_index = df_cis_on.last_valid_index()
    miles_remaining = total_miles_list[j]

    # Create '.kml' files for each CIS survey
    while miles_remaining > 0:
        kml = simplekml.Kml()
        style = simplekml.Style()
        # Create 3D data points
        while i <= last_index and df_cis_on.loc[i, 'Station'] < feet_counter:
            pnt = kml.newpoint(name=df_cis_on.loc[i, 'On Potential']*(-1))
            pnt.style.balloonstyle.text = \
                'Potential: -{} V'.format(df_cis_on.loc[i, 'On Potential'])
            pnt.coords = [(df_cis_on.loc[i, 'Longitude'],
                           df_cis_on.loc[i, 'Latitude'],
                           df_cis_on.loc[i, 'On Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = \
                'https://maps.google.com/mapfiles/kml/paddle/red-square-lv.png'
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 1
            i += 1

        kml.save(root_path + r'/Output/' +
             filtered_data_list[j].split('.DAT', 1)[0] + '/' +
             'Mile ' + kml_mile.__str__() + ' - ' + 'On.kml')
        kml_mile += 1
        miles_remaining -= 1

        if 0 < miles_remaining < 1:
            kml_mile = total_miles_list[j]

        feet_counter += 5280

    # Reset values for next CIS survey
    feet_counter = 5280
    kml_mile = 1
    j += 1

## Off

In [74]:
kml = simplekml.Kml()

In [75]:
df_cis_off = df_cis[df_cis['PS_OFF'] != 0]
df_cis_off = df_cis_off.iloc[:, [False, False, True, False, True, True, False]]
df_cis_off = df_cis_off[['LONGITUDE', 'LATITUDE', 'PS_OFF']].reset_index(drop=True)
df_cis_off = df_cis_off.iloc[:2112, :]
df_cis_off['PS_OFF'] = df_cis_off['PS_OFF'] * -1

IndexError: Boolean index has wrong length: 7 instead of 6

In [ ]:
# Counters
i = 0                                   # Row
z = df_cis_off.last_valid_index()       # Max iterations

while i < z:
    pnt = kml.newpoint(name=df_cis_off.loc[i, 'PS_OFF']*-1)
    pnt.style.balloonstyle.text = 'Potential: -{} V'.format(df_cis_off.loc[i, 'PS_OFF'])
    pnt.coords = [(df_cis_off.loc[i, 'LONGITUDE'], df_cis_off.loc[i, 'LATITUDE'], df_cis_off.loc[i, 'PS_OFF']*scale_factor)]

    # noinspection SpellCheckingInspection
    pnt.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/red-blank.png'
    pnt.style.iconstyle.scale = icon_scale
    pnt.style.labelstyle.scale = 0

    i += 1

kml.save(root_path + r'\Output/' + 'OFF - ' + project_name + '.kml')

## 1200

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_1200 = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_1200['Potential (V)'] = 1.200

In [ ]:
# Counters
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Potential: -{} V'.format(df_cis_1200.loc[i, 'Potential (V)'])
    ls.coords = [(df_cis_1200.loc[i, 'LONGITUDE'], df_cis_1200.loc[i, 'LATITUDE'], df_cis_1200.loc[i, 'Potential (V)']*scale_factor),
                 (df_cis_1200.loc[i+1, 'LONGITUDE'], df_cis_1200.loc[i+1, 'LATITUDE'], df_cis_1200.loc[i+1, 'Potential (V)']*scale_factor)] 
    ls.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)

    i += 1

kml.save(root_path + r'\Output/' + '1.2 V - ' + project_name + '.kml')

## 850

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_850 = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_850['Potential (V)'] = 0.850

In [ ]:
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Potential: {} V'.format(df_cis_850.loc[i, 'Potential (V)'])
    ls.coords = [(df_cis_850.loc[i, 'LONGITUDE'], df_cis_850.loc[i, 'LATITUDE'], df_cis_850.loc[i, 'Potential (V)']*scale_factor),
                 (df_cis_850.loc[i+1, 'LONGITUDE'], df_cis_850.loc[i+1, 'LATITUDE'], df_cis_850.loc[i+1, 'Potential (V)']*scale_factor)]
    ls.altitudemode = simplekml.AltitudeMode.relativetoground

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(175, 175, 175, 255)

    i += 1

kml.save(root_path + r'\Output/' + '0.85 V - ' + project_name + '.kml')

## Pipe

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_pipe = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
df_cis_pipe['Potential (V)'] = 0

In [ ]:
# Counters
i = 0                                    # Row
z = 282                                  # Max iterations

while i < z:
    ls = kml.newlinestring()
    ls.style.balloonstyle.text = 'Ground'
    ls.coords = [(df_cis_pipe.loc[i, 'LONGITUDE'], df_cis_pipe.loc[i, 'LATITUDE'], df_cis_pipe.loc[i, 'Potential (V)']),
                 (df_cis_pipe.loc[i+1, 'LONGITUDE'], df_cis_pipe.loc[i+1, 'LATITUDE'], df_cis_pipe.loc[i+1, 'Potential (V)'])]

    # Style
    ls.style.linestyle.width = 5
    ls.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, 175)

    i += 1

kml.save(root_path + r'\Output/' + 'Ground - ' + project_name + '.kml')

## Comments

In [ ]:
kml = simplekml.Kml()

In [ ]:
df_cis_comments = df_cis.iloc[:, [False, False, False, True, True, True, False]].copy()

In [ ]:
# Counters
i = 0                                  # Row
z = df_cis_comments.last_valid_index()       # Max iterations

while i < z:
    # noinspection SpellCheckingInspection
    if pd.isna(df_cis_comments.loc[i, 'COMMENTS']):
        i += 1

    else:
        pnt = kml.newpoint(name=df_cis_comments.loc[i, 'COMMENTS'])
        pnt.coords = [(df_cis_comments.loc[i, 'LONGITUDE'], df_cis_comments.loc[i, 'LATITUDE'], 0)]

        pnt.altitudemode = simplekml.AltitudeMode.relativetoground

        # Style
        pnt.style.iconstyle.icon.href = 'https://maps.google.com/mapfiles/kml/paddle/ltblu-blank.png'
        pnt.style.iconstyle.scale = icon_scale*0.05
        #pnt.style.labelstyle.scale = 0

        i += 1

kml.save(root_path + r'\Output/' + 'Comments - ' + project_name + '.kml')